## Series de tiempo


In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import pearsonr
import plotly.graph_objects as go
import plotly.express as px
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Output, Input
from statsmodels.tsa.holtwinters import ExponentialSmoothing

df = pd.read_csv("archivos_csv/bitcoin_usd.csv")
df.rename(columns={"Unnamed: 0":"date"},inplace=True)

df

### Análisis de serie temporal y forecasting

In [5]:
app = dash.Dash(__name__)

app.layout =  html.Div(id="body",className="e7_body",children=[
    html.H1("Análisis de tendencia y estacionalidad",id="title",style={"color":"white","font-family":"sans-serif"}),
    dcc.Dropdown(id="dropdown",style={"width":"175px","height":"25px","margin":"5px 0 25px 0"},
                        options=[
                            {"label":"Vólumen de trading","value":"volume"},
                            {"label":"Precio de apertura","value":"open_USD"},
                            {"label":"Precio de cierre","value":"close_USD"},
                            {"label":"Precio más bajo","value":"low_USD"},
                            {"label":"Precio más alto","value":"high_USD"},

                        ],
                        value="volume",
                        multi=False,
                        clearable=False),
    html.Div(id="dashboard",className="e7_dashboard",children=[
        dcc.Graph(id="trend",className="e7_graph",figure={}),
        dcc.Graph(id="correlogram",className="e7_graph",figure={})
    ]),
    html.H1("Forecasting",id="title_2",style={"text-align":"initial","color":"white","font-family":"sans-serif","margin":"15px 0"}),
    dcc.Graph(id="forecasting",figure={},style={"width":"100%","height":"500px","background-color":"white","margin":"25px 0"})
])

@app.callback(
    [Output(component_id="trend",component_property="figure"),
    Output(component_id="correlogram",component_property="figure"),
    Output(component_id="forecasting",component_property="figure")],
    [Input(component_id="dropdown",component_property="value")]
)

def update_graph(slct_var):
    
    levels = []
    period = list(range(0,df.shape[0]-5,5))

    for o in period:
        level = (df[slct_var][o] + df[slct_var][o+5]) / 2
        levels.append(level)

    df_period = df.loc[period,"date"]

    lineplot = go.Figure()
    lineplot.add_trace(go.Scatter(x=df["date"],y=df[slct_var], name="Serie", mode="lines"))
    lineplot.add_trace(go.Scatter(x=df_period,y=levels, name="Tendencia", line=dict(color='red', dash='dash')))
    lineplot.update_layout(title_text="Tendencia")
    lineplot.update_yaxes(title_text=slct_var)
    
    autocorr, _ = pearsonr(df[slct_var],df[slct_var])    
    autocorrelations = [autocorr]

    delay = -1
    recent = 1
    for r in list(range(0,df.shape[0]-2)):
        autocorr, _ = pearsonr(df[slct_var][recent:],df[slct_var][:delay])
        autocorrelations.append(autocorr)
        delay -= 1
        recent += 1
    
    delays = list(range(0,df.shape[0]-1))
    threshold = np.zeros(len(autocorrelations))

    correlogram = go.Figure()
    correlogram.add_trace(go.Scatter(x=delays,y=threshold,mode="lines",name="umbral"))
    correlogram.add_trace(go.Scatter(x=delays,y=autocorrelations,mode="markers",name="autocorrelación"))
    correlogram.update_layout(title_text="Estacionalidad")
    
    if slct_var == "volume":
        seasonal_periods = 100
    else:
        seasonal_periods = 230
    
    TES = ExponentialSmoothing(df[slct_var], trend="add", seasonal="add", seasonal_periods=seasonal_periods)
    model = TES.fit()
            
    predicts = model.forecast(30)
    
    forecasting = go.Figure()
    forecasting.add_trace(go.Scatter(x=df.index, y=df[slct_var], mode="lines", name="Serie"))
    forecasting.add_trace(go.Scatter(x=list(predicts.index), y=predicts, mode="lines", name="Pronóstico"))
    forecasting.update_layout(title_text="Pronóstico del siguiente mes")
    forecasting.update_xaxes(title_text='Período de 2018-05-07 a 2021-01-30')
    forecasting.update_yaxes(title_text=slct_var)
    

    return lineplot, correlogram, forecasting

if __name__ == "__main__":
    app.run_server(debug=False)

c:\Users\genar\AppData\Local\Programs\Python\Python312\Lib\site-packages\statsmodels\tsa\holtwinters\model.py:918: ConvergenceWarning:

Optimization failed to converge. Check mle_retvals.

